In [1]:
# GPU 런타임 설정을 한 후 실행한다.
!pip install konlpy

     |████████████████████████████████| 19.4MB 45.2MB/s 
     |████████████████████████████████| 460kB 48.6MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [3]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0    141      0  0:00:01  0:00:01 --:--:--   141
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 14.0M  100 14.0M    0     0  4112k      0  0:00:03  0:00:03 --:--:-- 11.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0    778      0 --:--:-- --:--:-- --:--:--   778
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4827k  100 4827k    0     0  1712k      0  0:00:02  0:00:02 --:--:-- 4561k


In [4]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [5]:
df_train[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [8]:
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()

twitter_tag = Twitter()


/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, 
                     max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

In [10]:
X_train.shape, y_train.shape

((150000, 3000), (150000,))

In [11]:
X_test.shape, y_test.shape

((50000, 3000), (50000,))

In [12]:
y_test[:3]

array([1, 0, 0])

# 로지스틱 회귀

In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.8406266666666666
Testing :  0.82616


# MLP

In [14]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [20]:
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])


In [26]:
model.fit(X_train.toarray(), y_train_cat,
        epochs=nb_epoch, 
        batch_size=batch_size) 

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.4512 - accuracy: 0.7787
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3474 - accuracy: 0.8425
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3258 - accuracy: 0.8553
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3010 - accuracy: 0.8709
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2675 - accuracy: 0.8891


In [28]:
y_pred = np.argmax(model.predict(X_test.toarray()), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.83038

# RNN

In [29]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)

(150000, 1, 3000)


In [30]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(128, input_shape=(X_train_rnn.shape[1], 
              X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [31]:
model.fit(X_train_rnn, y_train_cat,
          batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
1500/1500 [==============================] - 8s 4ms/step - loss: 0.4365 - accuracy: 0.7917
Epoch 2/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3418 - accuracy: 0.8452
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3027 - accuracy: 0.8677
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2480 - accuracy: 0.8978
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1928 - accuracy: 0.9218


In [32]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.82822

# LSTM

In [33]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X_train_rnn, y_train_cat,
          batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
1500/1500 [==============================] - 14s 6ms/step - loss: 0.4549 - accuracy: 0.7803
Epoch 2/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3540 - accuracy: 0.8370
Epoch 3/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3349 - accuracy: 0.8471
Epoch 4/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3047 - accuracy: 0.8653
Epoch 5/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2680 - accuracy: 0.8862


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.8339